In [ ]:
!pip install git+https://github.com/Bots-Avatar/inseq2 -q
!pip install git+https://github.com/Bots-Avatar/ExplainitAll -q

In [5]:
import gensim
from inseq import load_model

from explainitall import viz
from explainitall import dl
from explainitall import interp
import requests

In [6]:
def load_nlp_model(nlp_model_url):
    nlp_model_path = dl.DownloadManager.load_zip(nlp_model_url)
    return gensim.models.KeyedVectors.load_word2vec_format(nlp_model_path, binary=True)

# 'ID': 180
# 'Размер вектора': 300
# 'Корпус': 'Russian National Corpus'
# 'Размер словаря': 189193
# 'Алгоритм': 'Gensim Continuous Bag-of-Words'
# 'Лемматизация': True

nlp_model = load_nlp_model ('http://vectors.nlpl.eu/repository/20/180.zip')

Downloading: /root/.cache/180_zip: 100%|██████████| 462M/462M [00:27<00:00, 17.5MiB/s]
Extracting: /root/.cache/180_zip_data: 100%|██████████| 4/4 [00:09<00:00,  2.41s/it]


In [20]:
model_path = "sberbank-ai/rugpt3small_based_on_gpt2"

In [7]:
def load_gpt_model(gpt_model_name):
    return load_model(model=gpt_model_name,
                           attribution_method="integrated_gradients")

# 'Фреймворк': 'transformers'
# 'Тренировочные токены': '80 млрд'
# 'Размер контекста': 2048

gpt_model = load_gpt_model(model_path)

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

In [8]:
def value_interp(v):
  if str(v) == 'nan':
    return 'нулевой'
  if v < 0.1:
    return 'незначительной'
  if v < 0.3:
    return 'очень малой'
  if v < 0.45:
    return 'малой'
  if v < 0.65:
    return 'средней'
  if v < 0.85:
    return 'выше средней'
  else:
    return 'очень большой'

def interp_w(df):
  ret = ''
  for index, row in df.iterrows():
    for num_col, col in enumerate(df.columns):
        if num_col != 0:
          value = row[col]

          description = f'Слово "{row[df.columns[0]]}" влияет на генерацию слова "{col}" с {value_interp(value)} силой. \n'
          ret += description

  return ret + '.'

def interp_cl(df):
  ret = ''
  for index, row in df.iterrows():
    for num_col, col in enumerate(df.columns):
        if num_col != 0:
          value = row[col]

          description = f'Кластер "{row[df.columns[0]]}" влияет на генерацию кластера "{col}" с {value_interp(value)} силой. \n'
          ret += description

  return ret

In [9]:
clusters_discr = [
    {'name': 'Животные', 'centroid': ['собака', 'кошка', 'заяц'], 'top_k': 140},
    {'name': 'Лекарства', 'centroid': ['уколы', 'таблетки', 'микстуры'], 'top_k': 160},
    {'name': 'Болезни', 'centroid': ['простуда', 'орви', 'орз', 'грипп'], 'top_k': 20},
    {'name': 'Аллергия', 'centroid': ['аллергия'], 'top_k': 20}
]

explainer = interp.ExplainerGPT2(gpt_model=gpt_model, nlp_model=nlp_model)


input_text = 'у кошки грипп и аллергия на антибиотбиотики вопрос: чем лечить кошку? ответ:'
generated_text = 'лечите ее уколами'

expl_data = explainer.interpret(
    input_texts=input_text,
    generated_texts=generated_text,
    clusters_description=clusters_discr,
    batch_size=50,
    steps=34,
    # max_new_tokens=19
)

Attributing with integrated_gradients...: 100%|██████████| 26/26 [00:07<00:00,  1.57s/it]


In [10]:
def upload_file_to_fractalgpt(api_key, index_name, document_type, document_name, text):
    url = 'https://fractalgpt.ru/api/qa/upload'
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json'
    }

    payload = {
        "index_name": index_name,
        "document_type": document_type,
        "document_name": document_name,
        "data": list(text.encode('utf-8'))
    }

    response = requests.post(url, headers=headers, json=payload)
    return response

def get_answer_from_fractalgpt(api_key, index_name, question):
    url = 'https://fractalgpt.ru/api/qa/get-answer'
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json'
    }
    payload = {
        "index_name": index_name,
        "question": question
    }

    response = requests.post(url, headers=headers, json=payload)
    return response

def delete_index_fractalgpt(api_key, index_name):
    url = f'https://fractalgpt.ru/api/qa/delete?index_name={index_name}'
    headers = {
        'Authorization': f'Bearer {api_key}'
    }

    response = requests.delete(url, headers=headers)
    return response

In [28]:
# Результат интерпретации
imp_df = expl_data.word_imp_norm_df
imp_df_cl = expl_data.cluster_imp_aggr_df
w_desc = interp_w(imp_df)
cl_desc = interp_cl(imp_df_cl)
text = w_desc + cl_desc

data = f'Интерпретируемая фраза: {input_text} {generated_text}\n'
data += f"Интерпретируемая модель: {model_path}\n"
text = data+f'\n\nРезультат интерпретации:\n\n{{\n{text}\n}}'

In [29]:
print(text)

Интерпретируемая фраза: у кошки грипп и аллергия на антибиотбиотики вопрос: чем лечить кошку? ответ: лечите ее уколами
Интерпретируемая модель: sberbank-ai/rugpt3small_based_on_gpt2


Результат интерпретации:

{
Слово "у" влияет на генерацию слова "лечите" с средней силой. 
Слово "у" влияет на генерацию слова "ее" с малой силой. 
Слово "у" влияет на генерацию слова "уколами" с средней силой. 
Слово "кошки" влияет на генерацию слова "лечите" с очень малой силой. 
Слово "кошки" влияет на генерацию слова "ее" с очень малой силой. 
Слово "кошки" влияет на генерацию слова "уколами" с малой силой. 
Слово "грипп" влияет на генерацию слова "лечите" с малой силой. 
Слово "грипп" влияет на генерацию слова "ее" с малой силой. 
Слово "грипп" влияет на генерацию слова "уколами" с выше средней силой. 
Слово "и" влияет на генерацию слова "лечите" с незначительной силой. 
Слово "и" влияет на генерацию слова "ее" с незначительной силой. 
Слово "и" влияет на генерацию слова "уколами" с средней силой. 
С

In [23]:
api = 'API_KEY'

In [30]:
delete_index_fractalgpt(api, 'expl_data')

<Response [200]>

In [31]:
upload_file_to_fractalgpt(api, 'expl_data', 2, 'dat', text)

<Response [200]>

In [32]:
data_json = get_answer_from_fractalgpt(api, 'expl_data', 'Насколько важен кластер тип животного при генерации кластера лекарства?').json()
print(data_json['answer'])

Информация из документа "dat" говорит о том, что при генерации кластера лекарства важен кластер типа животного. Кластер "Животные" влияет на генерацию кластера "Лекарства" с очень большой силой.
